In [2]:
import flwr as fl
import sys
import numpy as np
import random
import string
import socket
from noknow.core import ZK, ZKSignature, ZKParameters, ZKData, ZKProof

clients=[]
clientsigs=[]

def registeration():
    password = random.randint(0,10)
    label = string.ascii_lowercase
    rand_string = ''.join(random.choice(label) for i in range(2))
    zk = ZK.new(curve_name="secp384r1", hash_alg="sha3_512")
    signature: ZKSignature = zk.create_signature(rand_string)
                                                            #         clientsig= ZK.sign(signature, rand_string)
    cL=str(signature)
    clientsigs.append(cL)
#     print(cL)
    encryptl=cL.encode('utf-8')
    return encryptl
SERVER_IP = '192.168.20.1'
SERVER_PORT = 5678
sig=''
with socket.socket(socket.AF_INET , socket.SOCK_STREAM) as s:
    x=0
    x+=1
    s.bind((SERVER_IP, SERVER_PORT))
    print('Server is listening')
    s.listen(x)
    conn,addr = s.accept()
    print(f'Connection accepted from :{addr}')
    with conn:
        while(True):
#             conn.send(b'Hello World')
#             data = conn.recv(1024)
#             data=data.decode('utf-8')
#             print(data)
            conn.send(registeration())
            client = conn.recv(1024)
            client=client.decode('utf-8')
            clients.append(client)
            break
with socket.socket(socket.AF_INET , socket.SOCK_STREAM) as s:
    x=0
    x+=1
    s.bind((SERVER_IP, SERVER_PORT))
    s.listen(x)
    conn,addr = s.accept()
    print(f'authentication:{addr}')
    with conn:
        while(True):
#             conn.send(b'send the secret please')
            data = conn.recv(1024)
            data=data.decode('utf-8')
            sig=data
#             print(sig)
            break
def authenticate(sig,i):
    if clientsigs[i]==sig:
        return True
    return False
for i in range(0,len(clients)):
    name=clients[i]
    print(name)
    if authenticate(sig,i):
        class SaveModelStrategy(fl.server.strategy.FedAvg):
            def aggregate_fit(
                self,
                rnd,
                results,
                failures
            ):
                aggregated_weights = super().aggregate_fit(rnd, results, failures)
                if aggregated_weights is not None:
#                     print(results)
                    # Save aggregated_weights
                    print(f"Saving round {rnd} aggregated_weights...")
                    np.savez(f"round-{rnd}-weights.npz", *aggregated_weights)
                    loss, accuracy = model.evaluate(super(), verbose=0)
                    print("Eval accuracy : ", accuracy)
                return aggregated_weights

        # Create strategy and run server
        strategy = SaveModelStrategy()

        # Start Flower server for three rounds of federated learning

        fl.server.start_server(
                server_address ="localhost:"+str(sys.argv[1]) , 
                grpc_max_message_length = 1024*1024*1024,
                strategy = strategy
        )

    else:
        print("malicious user")

Server is listening
Connection accepted from :('192.168.20.1', 54259)


INFO flower 2022-11-28 15:46:27,916 | app.py:119 | Starting Flower server, config: ServerConfig(num_rounds=1, round_timeout=None)


authentication:('192.168.20.1', 54260)
client2


RuntimeError: Failed to bind to address localhost:-f; set GRPC_VERBOSITY=debug environment variable to see detailed error message.